# Laden der Daten
In diesem Notebook werden die Daten eingelesen, welche direkt über die URL zugängig sind. Die Geodaten für das Weg- und Velonetz <br> 
in Zürich müssen manuell auf der Open Data Seite runtergeladen und dann über einen CSV-Import geladen werden, nachdem die Datei Entzipped wurde.<br>
Grund hierfür ist die Auswahl des Ausschnitts, in welchem das Velonetz angezeigt werden soll. <br>
Dieser muss auf der Seite ausgewählt werden und es wird dann ein Downlaodlink für 5 Minuten zur Verfügung gestellt.

## Verwendete Links/Quellen

Letzter Zugriff: 01.12.2021

In [1]:
url_5g="http://data.geo.admin.ch/ch.bakom.mobil-antennenstandorte-5g/data/ch.bakom.mobil-antennenstandorte-5g_de.json"
url_lte='http://data.geo.admin.ch/ch.bakom.mobil-antennenstandorte-lte/data/ch.bakom.mobil-antennenstandorte-lte_de.json'

***

# Installation notwendiger Bibliotheken

In [2]:
!pip install --upgrade pip
!pip install folium
!pip install matplotlib
!pip install geopandas
!pip install geopy
!pip install pyproj

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts.endress.com/artifactory/api/pypi/pypi-mirror/simple
     ---------------------------------------- 2.1/2.1 MB 10.1 MB/s eta 0:00:00


ERROR: To modify pip, please run the following command:
C:\Anaconda3\python.exe -m pip install --upgrade pip


Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts.endress.com/artifactory/api/pypi/pypi-mirror/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts.endress.com/artifactory/api/pypi/pypi-mirror/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts.endress.com/artifactory/api/pypi/pypi-mirror/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts.endress.com/artifactory/api/pypi/pypi-mirror/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts.endress.com/artifactory/api/pypi/pypi-mirror/simple


***

# Module importieren

In [3]:
import csv

# library to handle data in a vectorized manner 
import numpy as np 

# library for data analsysis
import pandas as pd

import geopandas as gpd

# library to handle JSON files
import json

# tranform JSON file into a pandas dataframe
from pandas.io.json import json_normalize

import folium
from folium import plugins

# library to handle requests
import requests

import zipfile

# convert an address into latitude and longitude values
from geopy.geocoders import Nominatim

# necessary for loading url data
import io


from pyproj import Proj, transform

print('Libraries imported.')


from shapely.geometry import Point, LineString

Libraries imported.


***

# Funktionen

In [4]:
# https://www.swisstopo.admin.ch/content/swisstopo-internet/de/online/calculation-services/_jcr_content/contentPar/tabs/items/dokumente_und_publik/tabPar/downloadlist/downloadItems/8_1467103085694.download/refsys_d.pdf)
# Zugriff: 01:11:2021/16:54
def convert_lv95_wgs84(e,n): #Funktion Umrechnung lv95 zu wgs84
    
    y = (e - 2600000)/1000000
    x = (n - 1200000)/1000000
    
    l = 2.6779094 + 4.728982 * y + 0.791484 * y * x + 0.1306 * y * x**2 - 0.0436 * y**3
    b = 16.9023892 + 3.238272 * x - 0.270978 * y**2 - 0.002528 * x**2 - 0.0447 * y**2 * x - 0.0140 * x**3
    
    laenge_wgs84 = l * 100 / 36
    breite_wgs84 = b * 100 / 36
    
    return breite_wgs84, laenge_wgs84

Die Klasse load_data wurde implementiert, um alle Daten mit einem Befehl einlesen zu können. <br>
Dies ist nicht unbedingt notwendig, da das Laden mit einer Codezeile erfolgen kann, diente aber als Übung.<br>
Noch dazu können andere Files, je nach Art, geladen werden.

In [5]:
class load_data:
    def __init__(self,path):
        self.path=path

    @staticmethod
    def load_from_url(url):
        s=requests.get(url).content
        df=pd.read_csv(io.StringIO(s.decode('utf-8')))
        return df

        
    @staticmethod
    def load_pkl(path_to_pkl):
        with open(path_to_pkl,'rb') as f:
            data=pickle.load(f)
        return data
        
    @staticmethod
    def load_csv(path_to_csv):
        df=pd.read_csv(path_to_csv,sep=',')
        return df

***

# 1) Polizeilich registrierte Verkehrsunfälle im Stadtgebiet Zürich seit 2011
<br>Dieser Datensatz liefert die polizeilich registrierten Strassenverkehrsunfälle mit Personen- und Sachschäden auf dem Stadtgebiet Zürich seit 2011<br/>

In [6]:
data_path='./Daten/ch.bakom.mobil-antennenstandorte-5g_de.json'
antennen_5g = gpd.read_file(data_path)
antennen_5g.head()

,lang,description,id,powercode_de,geometry
0,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",1,Mittel,POINT (2688674.000 1211488.000)
1,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",2,Sehr Klein,POINT (2666065.000 1214355.000)
2,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",3,Mittel,POINT (2722651.000 1271244.000)
3,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",4,Mittel,POINT (2570323.000 1164067.000)
4,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",5,Klein,POINT (2613427.000 1262637.000)


In [7]:

columns=['language', 'description', 'id', 'power', 'cordinates']
antennen_5g.columns=columns

In [30]:
antennen_5g.drop(['language' , 'description'], axis=1, inplace=True)
antennen_5g.head(4)

,id,power,cordinates
0,1,Mittel,POINT (2688674.000 1211488.000)
1,2,Sehr Klein,POINT (2666065.000 1214355.000)
2,3,Mittel,POINT (2722651.000 1271244.000)
3,4,Mittel,POINT (2570323.000 1164067.000)


In [17]:
print(antennen_5g['cordinates'].dtype)
antennen_5g['cordinates'][1].x

geometry


2666065.0

In [18]:
# LV95 zu WGS84
inProj = Proj('epsg:2056')
outProj = Proj('epsg:4326')
x1,y1 = antennen_5g['cordinates'].x, antennen_5g['cordinates'].y
x2,y2 = transform(inProj, outProj, x1,y1)

C:\Users\i00202914\AppData\Local\Temp\ipykernel_11536\1161446565.py:5: FutureWarning: This function is deprecated. See: https://pyproj4.github.io/pyproj/stable/gotchas.html#upgrading-to-pyproj-2-from-pyproj-1
  x2,y2 = transform(inProj, outProj, x1,y1)


In [53]:
import ssl
!pip install certifi
import certifi
print(certifi.where())
import geopy.geocoders
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
print(certifi.where())

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts.endress.com/artifactory/api/pypi/pypi-mirror/simple
C:\Anaconda3\lib\site-packages\certifi\cacert.pem
C:\Anaconda3\lib\site-packages\certifi\cacert.pem


In [49]:
antennen_5g['AccidentLocation_WGS84_E'] = x2
antennen_5g['AccidentLocation_WGS84_N'] = y2
antennen_5g.head()

,language,description,id,power,cordinates,AccidentLocation_WGS84_E,AccidentLocation_WGS84_N
0,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",1,Mittel,POINT (2688674.000 1211488.000),47.048492,8.605623
1,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",2,Sehr Klein,POINT (2666065.000 1214355.000),47.076917,8.308525
2,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",3,Mittel,POINT (2722651.000 1271244.000),47.580429,9.069008
3,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",4,Mittel,POINT (2570323.000 1164067.000),46.627189,7.051134
4,de,"<table>\n<tr><td class=\""cell-left\"">\nSendele...",5,Klein,POINT (2613427.000 1262637.000),47.514341,7.616871


In [59]:
from geopy.geocoders import Nominatim
!pip install certifi
!pip install ssl
import ssl
import certifi
print(certifi.where())
import geopy.geocoders
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
print(certifi.where())
geolocator = Nominatim(user_agent="test")
location = geolocator.reverse("52.509669, 13.376294")

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts.endress.com/artifactory/api/pypi/pypi-mirror/simple
Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://artifacts.endress.com/artifactory/api/pypi/pypi-mirror/simple
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'error'
C:\Anaconda3\lib\site-packages\certifi\cacert.pem
C:\Anaconda3\lib\site-packages\certifi\cacert.pem


  error: subprocess-exited-with-error
  
  python setup.py egg_info did not run successfully.
  exit code: 1
  
  [25 lines of output]
  Traceback (most recent call last):
    File "<string>", line 2, in <module>
    File "<pip-setuptools-caller>", line 14, in <module>
    File "C:\Anaconda3\lib\site-packages\setuptools\__init__.py", line 189, in <module>
      monkey.patch_all()
    File "C:\Anaconda3\lib\site-packages\setuptools\monkey.py", line 99, in patch_all
      patch_for_msvc_specialized_compiler()
    File "C:\Anaconda3\lib\site-packages\setuptools\monkey.py", line 169, in patch_for_msvc_specialized_compiler
      patch_func(*msvc14('_get_vc_env'))
    File "C:\Anaconda3\lib\site-packages\setuptools\monkey.py", line 149, in patch_params
      mod = import_module(mod_name)
    File "C:\Anaconda3\lib\importlib\__init__.py", line 127, in import_module
      return _bootstrap._gcd_import(name[level:], package, level)
    File "C:\Anaconda3\lib\site-packages\setuptools\_distutils\

GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=52.509669&lon=13.376294&format=json&addressdetails=1 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1129)')))

In [45]:
from geopy.exc import GeocoderTimedOut
import ssl

context = ssl.create_default_context()
der_certs = context.get_ca_certs(binary_form=True)
pem_certs = [ssl.DER_cert_to_PEM_cert(der) for der in der_certs]

with open('wincacerts.pem', 'w') as outfile:
    for pem in pem_certs:
        outfile.write(pem + '\n')
print outfile.where())
def get_kreis(coordinates, attempt=1, max_attempts=5):
    try:
        locator = Nominatim(user_agent="1_Dataset_Load_cleansing")

        location = locator.reverse(coordinates)
     #   return location.address.split(",")[-6].strip() if len(location.address.split(",")) > 6 else location.address.split(",")[-5].strip()
    except GeocoderTimedOut:
        if attempt <= max_attempts:
            return get_kreis(coordinates, attempt=attempt+1)
        raise

SyntaxError: invalid syntax (3412181519.py, line 11)

In [34]:
def get_quartier(coordinates, attempt=1, max_attempts=5):
    try:
        locator = Nominatim(user_agent="1_Dataset_Load_cleansing")
        location = locator.reverse(coordinates)
        #print(len(location.address.split(",")), " - ", location.address.split(","))
        return location.address.split(",")[-7].strip() if len(location.address.split(",")) > 6 else location.address.split(",")[-6].strip()
    except GeocoderTimedOut:
        if attempt <= max_attempts:
            return get_kreis(coordinates, attempt=attempt+1)
        raise

In [35]:
for index, row in antennen_5g.iterrows():
    print(row['AccidentLocation_WGS84_E'])

47.04849186405694
47.07691745412742
47.580429226152305
46.6271888972478
47.514340719360185
47.2713022762742
47.59191660026886
46.22072263214226
47.23968808113957
46.52715091030971
47.096464682126985
46.18721217870401
47.48235402873181
47.389118842273945
47.39425448564568
47.40466956825127
46.06772160244409
46.71637744880908
46.936643130301306
46.86615740298081
46.617884713185575
47.380210762287
46.974817216364045
46.50062402327223
47.65273681003993
47.479219738969434
46.82059413431265
47.36914532275527
46.206362871420865
47.35397215126742
47.41537423535369
46.27890914294518
46.20626719826441
45.988622949956635
46.903311435504
46.26426737399861
46.95466143968786
46.68612889179624
47.2110351707968
46.709078664975735
46.6173372799504
46.21190881799157
46.34714831252968
47.33753998233614
46.54196476318328
47.05621515810808
47.42486565126919
46.799989176753584
47.216178694705825
46.52819950546847
46.603463617914436
47.410906399018536
47.35470001127684
46.99887347604012
46.19663341648916
47.

In [46]:
kreis = []
quartier = []
for index, row in antennen_5g.iterrows():
    print(row['AccidentLocation_WGS84_E'])
    coordinates = [row['AccidentLocation_WGS84_E'],row["AccidentLocation_WGS84_N"]]
    cor = get_kreis(coordinates)
    print(cor)
    kreis.append(get_kreis(coordinates))
    quartier.append(get_quartier(coordinates))
    print("|", end = '' if index % 100 != 0 else '\n')
    break

47.04849186405694


GeocoderUnavailable: HTTPSConnectionPool(host='nominatim.openstreetmap.org', port=443): Max retries exceeded with url: /reverse?lat=47.04849186405694&lon=8.605623335590286&format=json&addressdetails=1 (Caused by SSLError(SSLCertVerificationError(1, '[SSL: CERTIFICATE_VERIFY_FAILED] certificate verify failed: self signed certificate in certificate chain (_ssl.c:1129)')))

In [ ]:
df_unfaelle=load_data.load_from_url(url_unfaelle)
df_unfaelle.head()

In [ ]:
## 1.5) Reverse Geocoding - Quartier & Stadtkreis

In [29]:
# mail von Maurizio, 24.11.21, workaround for SSL Error:
import ssl
import certifi
import geopy.geocoders
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
print(certifi.where())

C:\Anaconda3\lib\site-packages\certifi\cacert.pem


In [ ]:
def get_kreis(coordinates, attempt=1, max_attempts=5):
    try:
        locator = Nominatim(user_agent="1_Dataset_Load_cleansing")
        location = locator.reverse(coordinates)
        return location.address.split(",")[-6].strip() if len(location.address.split(",")) > 6 else location.address.split(",")[-5].strip()
    except GeocoderTimedOut:
        if attempt <= max_attempts:
            return get_kreis(coordinates, attempt=attempt+1)
        raise

In [ ]:
def get_quartier(coordinates, attempt=1, max_attempts=5):
    try:
        locator = Nominatim(user_agent="1_Dataset_Load_cleansing")
        location = locator.reverse(coordinates)
        #print(len(location.address.split(",")), " - ", location.address.split(","))
        return location.address.split(",")[-7].strip() if len(location.address.split(",")) > 6 else location.address.split(",")[-6].strip()
    except GeocoderTimedOut:
        if attempt <= max_attempts:
            return get_kreis(coordinates, attempt=attempt+1)
        raise

In [30]:
kreis = []
quartier = []
for index, row in df_verkehrsunfaelle_2020.iterrows():
    coordinates = [row["AccidentLocation_WGS84_E"],row["AccidentLocation_WGS84_N"]]
    kreis.append(get_kreis(coordinates))
    quartier.append(get_quartier(coordinates))
    print("|", end = '' if index % 100 != 0 else '\n')

NameError: name 'df_verkehrsunfaelle_2020' is not defined

In [ ]:
# Einige der Kreise wurden nicht gefunden und muss mit Hilfe der Lookup Tabelle bereinigt werden
kreis_lookup = {
    'Wiedikon': 'Kreis 3',
    'Schwamendingen': 'Kreis 12',
    'Riesbach': 'Kreis 8',
    'Altstadt': 'Kreis 1',
    'Aussersihl': 'Kreis 4',
    'Industriequartier': 'Kreis 5',
    'Hochschulen': 'Kreis 1',
    'Escher Wyss': 'Kreis 5',
    'Seebach': 'Kreis 11',
    'Pfaffhausen': 'Kreis 7'
}

Im Folgenden werden die ersten Reinigunsschritte getätigt. Die Angaben der Unfaelle sind auf 4 verschiedenen Sprachen <br> 
verfügbar. Es werden allerdings nur die Hauptspalten benötigt, weswegen die anderen Sprachen fallen gelassen werden können. <br> 
Die 'AccidentType_de' Spalte wird allrdings einer neuen Spalte zugewiesen, da die dort enthalten Informationen <br> 
wichtig sind und nur umbenannt.<br><br>

Uns interessieren ausserdem nur die Unfaelle die in 2019 vorkamen, sowie Unfaelle in die ein Fahrrad<br>
involviert war. Diese werden gefiltert und die Umlaute in der 'AccidentExpl' ersetzt.

In [34]:
# Überflüssige Spalten löschen
try:
    df_unfaelle=df_unfaelle.rename({'AccidentType_de':'AccidentExpl'},axis=1)
    df_unfaelle['AccidentWeekDay']=df_unfaelle['AccidentWeekDay_de']
    df_unfaelle['RoadType']=df_unfaelle['RoadType_de']
    df_unfaelle['AccidentSeverityCategory']=df_unfaelle.AccidentSeverityCategory_de
    df_unfaelle=df_unfaelle.drop(columns=list(df_unfaelle.filter(regex='(_fr|_it|_en|_de|Code|text|UID)')))
    df_unfaelle=df_unfaelle.drop(columns='AccidentType')

except:
    print('Already Dropped')

In [35]:
# Filtern nach Jahr == 2019 und Velofahrer in Unfall verwickelt == True
df_unfaelle = df_unfaelle[(df_unfaelle.AccidentInvolvingBicycle == True) & (df_unfaelle.AccidentYear == 2019)]

In [36]:
# Umschreiben der Umlaute in ae usw.
df_unfaelle=df_unfaelle.replace({'AccidentExpl':{'ä':'ae','Ä':'Ae','Ü':'Ue','ü':'ue','Ö':'Oe','ö':'oe'}},regex=True)

In [37]:
# Index Reset
df_unfaelle.reset_index(inplace=True,drop=True)
df_unfaelle.head()

,AccidentExpl,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,AccidentYear,AccidentMonth,AccidentWeekDay,AccidentHour
0,Frontalkollision,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2682347,1247786,2019,1,Freitag,13
1,Einbiegeunfall,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2680457,1247471,2019,1,Montag,6
2,Ueberqueren der Fahrbahn,Unfall mit Sachschaden,False,True,False,Hauptstrasse,2681804,1246614,2019,1,Montag,18
3,Abbiegeunfall,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2682226,1247798,2019,1,Dienstag,7
4,Ueberqueren der Fahrbahn,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2682856,1247821,2019,1,Dienstag,19


In [38]:
df_unfaelle.isnull().values.any() # Übeprüfung missing values

False

Umwandeln der Koordinaten mithilfe der Funktion.

In [39]:
df_unfaelle['Latidute_wgs84'],df_unfaelle['Longitude_wgs84'] = zip(*df_unfaelle.apply(lambda x: convert_lv95_wgs84(x.AccidentLocation_CHLV95_E,x.AccidentLocation_CHLV95_N), axis=1))
df_unfaelle

,AccidentExpl,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,AccidentYear,AccidentMonth,AccidentWeekDay,AccidentHour,Latidute_wgs84,Longitude_wgs84
0,Frontalkollision,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2682347,1247786,2019,1,Freitag,13,47.375761,8.529004
1,Einbiegeunfall,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2680457,1247471,2019,1,Montag,6,47.373162,8.503925
2,Ueberqueren der Fahrbahn,Unfall mit Sachschaden,False,True,False,Hauptstrasse,2681804,1246614,2019,1,Montag,18,47.365288,8.521601
3,Abbiegeunfall,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2682226,1247798,2019,1,Dienstag,7,47.375884,8.527405
4,Ueberqueren der Fahrbahn,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2682856,1247821,2019,1,Dienstag,19,47.376012,8.535750
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
690,Ueberholunfall oder Fahrstreifenwechsel,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2682340,1244588,2019,12,Freitag,12,47.347001,8.528324
691,Abbiegeunfall,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2682804,1247769,2019,12,Sonntag,4,47.375551,8.535052
692,Ueberqueren der Fahrbahn,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2680387,1247556,2019,12,Sonntag,19,47.373935,8.503014
693,Schleuder- oder Selbstunfall,Unfall mit Sachschaden,False,True,True,Nebenstrasse,2683523,1247624,2019,12,Dienstag,12,47.374156,8.544544


Einlesen als Geopandas DataFrame und umwandeln der Koordinaten mithilfe <br>
von Geopandas in eine Geometrie.

In [40]:
from shapely.geometry import Point
geometry = [Point(xy) for xy in zip(df_unfaelle.AccidentLocation_CHLV95_E, df_unfaelle.AccidentLocation_CHLV95_N)]
gdf_unfaelle = gpd.GeoDataFrame(df_unfaelle, geometry=geometry,crs='EPSG:2056')
gdf_unfaelle=gdf_unfaelle.to_crs(epsg=4326)
gdf_unfaelle.head(5)

,AccidentExpl,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,AccidentLocation_CHLV95_E,AccidentLocation_CHLV95_N,AccidentYear,AccidentMonth,AccidentWeekDay,AccidentHour,Latidute_wgs84,Longitude_wgs84,geometry
0,Frontalkollision,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2682347,1247786,2019,1,Freitag,13,47.375761,8.529004,POINT (8.52901 47.37576)
1,Einbiegeunfall,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2680457,1247471,2019,1,Montag,6,47.373162,8.503925,POINT (8.50393 47.37317)
2,Ueberqueren der Fahrbahn,Unfall mit Sachschaden,False,True,False,Hauptstrasse,2681804,1246614,2019,1,Montag,18,47.365288,8.521601,POINT (8.52160 47.36529)
3,Abbiegeunfall,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2682226,1247798,2019,1,Dienstag,7,47.375884,8.527405,POINT (8.52741 47.37589)
4,Ueberqueren der Fahrbahn,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2682856,1247821,2019,1,Dienstag,19,47.376012,8.535750,POINT (8.53575 47.37602)


In [41]:
# Überflüssige Spalten löschen
gdf_unfaelle.drop(gdf_unfaelle.filter(regex=r'(CHLV95)').columns, axis=1, inplace=True)
gdf_unfaelle.head(5)

,AccidentExpl,AccidentSeverityCategory,AccidentInvolvingPedestrian,AccidentInvolvingBicycle,AccidentInvolvingMotorcycle,RoadType,AccidentYear,AccidentMonth,AccidentWeekDay,AccidentHour,Latidute_wgs84,Longitude_wgs84,geometry
0,Frontalkollision,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2019,1,Freitag,13,47.375761,8.529004,POINT (8.52901 47.37576)
1,Einbiegeunfall,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2019,1,Montag,6,47.373162,8.503925,POINT (8.50393 47.37317)
2,Ueberqueren der Fahrbahn,Unfall mit Sachschaden,False,True,False,Hauptstrasse,2019,1,Montag,18,47.365288,8.521601,POINT (8.52160 47.36529)
3,Abbiegeunfall,Unfall mit Sachschaden,False,True,False,Nebenstrasse,2019,1,Dienstag,7,47.375884,8.527405,POINT (8.52741 47.37589)
4,Ueberqueren der Fahrbahn,Unfall mit Leichtverletzten,False,True,False,Nebenstrasse,2019,1,Dienstag,19,47.376012,8.535750,POINT (8.53575 47.37602)


In [42]:
gdf_unfaelle.isnull().values.any() # Übeprüfung missing values

False

In [43]:
gdf_unfaelle.to_csv('./Daten/velounfaelle_2019_ZH.csv',sep=';', index=False)

***

# 2) Daten der automatischen Fussgänger- und Velozählung - Viertelstundenwerte der Stadt Zürich 2019
<br>Die Daten zeigen die gezählten Frequenzen je Viertelstunde und Richtung<br/>

In [44]:
df_verkehrszaehlung = load_data.load_from_url(url_zaehl)
df_verkehrszaehlung.head()

,FK_ZAEHLER,FK_STANDORT,DATUM,VELO_IN,VELO_OUT,FUSS_IN,FUSS_OUT,OST,NORD
0,Y2G14045587,61,2019-01-01T00:00,0.0,NaN,NaN,NaN,2683447,1247063
1,U15G3063872,32,2019-01-01T00:00,NaN,NaN,0.0,15.0,2683511,1248166
2,ECO09113502,9,2019-01-01T00:00,0.0,0.0,NaN,NaN,2684578,1251967
3,ECO06040592,3918,2019-01-01T00:00,4.0,6.0,NaN,NaN,2683573,1248545
4,U15G4025180,3278,2019-01-01T00:00,NaN,NaN,1.0,4.0,2680439,1249930


Umbennen der Spaltennamen

In [45]:
columns=['zaehler', 'standort', 'datum', 'velo_in', 'velo_out', 'fuss_in',
       'fuss_out', 'ost', 'nord']
df_verkehrszaehlung.columns=columns
df_verkehrszaehlung.head()

,zaehler,standort,datum,velo_in,velo_out,fuss_in,fuss_out,ost,nord
0,Y2G14045587,61,2019-01-01T00:00,0.0,NaN,NaN,NaN,2683447,1247063
1,U15G3063872,32,2019-01-01T00:00,NaN,NaN,0.0,15.0,2683511,1248166
2,ECO09113502,9,2019-01-01T00:00,0.0,0.0,NaN,NaN,2684578,1251967
3,ECO06040592,3918,2019-01-01T00:00,4.0,6.0,NaN,NaN,2683573,1248545
4,U15G4025180,3278,2019-01-01T00:00,NaN,NaN,1.0,4.0,2680439,1249930


In [46]:
df_verkehrszaehlung['Latidute_wgs84'],df_verkehrszaehlung['Longitude_wgs84'] = zip(*df_verkehrszaehlung.apply(lambda x: convert_lv95_wgs84(x.ost,x.nord), axis=1))
#df_verkehrszaehlung

Das Dateiformat wird in ein Datetime format umgewandelt und dann die jeweiligen Informationen zu <br>
Jahr, Tag, Stunde,Minute in eigene Spalten geschrieben.

In [47]:
df_verkehrszaehlung['datum']=pd.to_datetime(df_verkehrszaehlung.datum)
df_verkehrszaehlung['year']=pd.to_datetime(df_verkehrszaehlung.datum).dt.year
df_verkehrszaehlung['month']=pd.to_datetime(df_verkehrszaehlung.datum).dt.month
df_verkehrszaehlung['day']=pd.to_datetime(df_verkehrszaehlung.datum).dt.day
df_verkehrszaehlung['hour']=pd.to_datetime(df_verkehrszaehlung.datum).dt.hour
df_verkehrszaehlung['minute']=pd.to_datetime(df_verkehrszaehlung.datum).dt.minute
df_verkehrszaehlung.head(4)

,zaehler,standort,datum,velo_in,velo_out,fuss_in,fuss_out,ost,nord,Latidute_wgs84,Longitude_wgs84,year,month,day,hour,minute
0,Y2G14045587,61,2019-01-01,0.0,NaN,NaN,NaN,2683447,1247063,47.369120,8.543433,2019,1,1,0,0
1,U15G3063872,32,2019-01-01,NaN,NaN,0.0,15.0,2683511,1248166,47.379032,8.544486,2019,1,1,0,0
2,ECO09113502,9,2019-01-01,0.0,0.0,NaN,NaN,2684578,1251967,47.413080,8.559332,2019,1,1,0,0
3,ECO06040592,3918,2019-01-01,4.0,6.0,NaN,NaN,2683573,1248545,47.382433,8.545378,2019,1,1,0,0


In [48]:
try:
    df_verkehrszaehlung.drop(['ost','nord'], axis=1, inplace=True)
except:
    print('already dropped')

In [49]:
df_verkehrszaehlung=df_verkehrszaehlung.fillna(0)
df_verkehrszaehlung.head(5)

,zaehler,standort,datum,velo_in,velo_out,fuss_in,fuss_out,Latidute_wgs84,Longitude_wgs84,year,month,day,hour,minute
0,Y2G14045587,61,2019-01-01,0.0,0.0,0.0,0.0,47.369120,8.543433,2019,1,1,0,0
1,U15G3063872,32,2019-01-01,0.0,0.0,0.0,15.0,47.379032,8.544486,2019,1,1,0,0
2,ECO09113502,9,2019-01-01,0.0,0.0,0.0,0.0,47.413080,8.559332,2019,1,1,0,0
3,ECO06040592,3918,2019-01-01,4.0,6.0,0.0,0.0,47.382433,8.545378,2019,1,1,0,0
4,U15G4025180,3278,2019-01-01,0.0,0.0,1.0,4.0,47.395279,8.504129,2019,1,1,0,0


Und umgewandelt als Geopandas mit geometry:

In [50]:
from shapely.geometry import Point

In [51]:
geometry = [Point(xy) for xy in zip(df_verkehrszaehlung.Longitude_wgs84, df_verkehrszaehlung.Latidute_wgs84)]

In [52]:
gdf_verkehrszaehlung = gpd.GeoDataFrame(df_verkehrszaehlung, geometry=geometry,crs='EPSG:4326')

In [53]:
gdf_verkehrszaehlung=gdf_verkehrszaehlung.to_crs(epsg=4326)

In [54]:
gdf_verkehrszaehlung.head(5)

,zaehler,standort,datum,velo_in,velo_out,fuss_in,fuss_out,Latidute_wgs84,Longitude_wgs84,year,month,day,hour,minute,geometry
0,Y2G14045587,61,2019-01-01,0.0,0.0,0.0,0.0,47.369120,8.543433,2019,1,1,0,0,POINT (8.54343 47.36912)
1,U15G3063872,32,2019-01-01,0.0,0.0,0.0,15.0,47.379032,8.544486,2019,1,1,0,0,POINT (8.54449 47.37903)
2,ECO09113502,9,2019-01-01,0.0,0.0,0.0,0.0,47.413080,8.559332,2019,1,1,0,0,POINT (8.55933 47.41308)
3,ECO06040592,3918,2019-01-01,4.0,6.0,0.0,0.0,47.382433,8.545378,2019,1,1,0,0,POINT (8.54538 47.38243)
4,U15G4025180,3278,2019-01-01,0.0,0.0,1.0,4.0,47.395279,8.504129,2019,1,1,0,0,POINT (8.50413 47.39528)


Abspeichern als csv:

In [55]:
gdf_verkehrszaehlung.to_csv('./Daten/verkehrszaehlung_2019_ZH.csv',sep=';', index=False)

***

# 3) Fuss und Fahrradwegnetz Stadt Zürich
<br>Das Fuss- und Velowegnetz beinhaltet alle Fuss- und Velowege in der Stadt Zürich. Es bildet die Grundlage für den städtischen Routenplaner.<br/>
<br>Zugriff: 01:11:2021/20:05 https://data.stadt-zuerich.ch/dataset/geo_fuss__und_velowegnetz<br/>

In [56]:
zip_ref = zipfile.ZipFile('./Daten/Fuss_Velo_Wegnetz.zip')
zip_ref.extractall('./Daten/')
zip_ref.close()

In [57]:
data_path='./Daten/Fuss_Velo_Wegnetz/data/taz_mm.tbl_routennetz.json'
gdf_routennetz = gpd.read_file(data_path)

In [58]:
gdf_routennetz.head()

,id1,velo,velostreifen,veloweg,einbahn,fuss,name,map_velo,map_fuss,se_anno_cad_data,objectid,geometry
0,137,0,0,0,0,1,"Bahnhof Oerlikon, Perron Gleis 6",0,0,NaN,1,"LINESTRING (8.54406 47.41199, 8.54443 47.41212)"
1,138,0,0,0,0,0,Bahnhofstrasse,0,0,NaN,2,"LINESTRING (8.53985 47.37686, 8.53986 47.37688..."
2,139,1,0,0,0,0,Zwischenbächen,1,0,NaN,3,"LINESTRING (8.48085 47.38451, 8.48058 47.38413)"
3,140,1,0,0,0,0,Schanzackerstrasse,0,0,NaN,4,"LINESTRING (8.54727 47.39015, 8.54757 47.39119..."
4,141,0,0,0,0,1,Rotbuchstrasse Überquerung,0,0,NaN,5,"LINESTRING (8.53742 47.39127, 8.53742 47.39141)"


In [59]:
gdf_routennetz.drop(['se_anno_cad_data','id1','objectid'], axis=1, inplace=True)
gdf_routennetz.head(4)

,velo,velostreifen,veloweg,einbahn,fuss,name,map_velo,map_fuss,geometry
0,0,0,0,0,1,"Bahnhof Oerlikon, Perron Gleis 6",0,0,"LINESTRING (8.54406 47.41199, 8.54443 47.41212)"
1,0,0,0,0,0,Bahnhofstrasse,0,0,"LINESTRING (8.53985 47.37686, 8.53986 47.37688..."
2,1,0,0,0,0,Zwischenbächen,1,0,"LINESTRING (8.48085 47.38451, 8.48058 47.38413)"
3,1,0,0,0,0,Schanzackerstrasse,0,0,"LINESTRING (8.54727 47.39015, 8.54757 47.39119..."


In [60]:
gdf_routennetz=gdf_routennetz.replace({'name':{'ä':'ae','ö':'oe','ü':'ue','Ü':'Ue','Ä':'Ae'}},regex=True)

In [61]:
gdf_routennetz.to_csv('./Daten/routennetz_ZH.csv',sep=';', index=False)

Erster Plot zum anschauen des Netzes:

In [62]:
# mail von Maurizio, 24.11.21, workaround for SSL Error:
import ssl
import certifi
import geopy.geocoders
ctx = ssl.create_default_context(cafile=certifi.where())
geopy.geocoders.options.default_ssl_context = ctx
print(certifi.where())

/opt/conda/lib/python3.10/site-packages/certifi/cacert.pem


In [71]:
#Breiten- und Längengrad von Zürich
address = 'Zurich, CH'

geolocator = Nominatim(user_agent="Jupyter")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
#print('The geograpical coordinate of Zurich, CH are {}, {}.'.format(latitude, longitude))

In [72]:
m=folium.Map(location=[latitude,longitude],zoom_start=12)

In [73]:
folium.Choropleth(gdf_routennetz[gdf_routennetz.velostreifen!='0'],line_color='blue',line_weight=5,line_opacity=0.3,columns=['name'],legend_name='hehe').add_to(m)
folium.Choropleth(gdf_routennetz[gdf_routennetz.veloweg!=0].geometry,line_color='red',line_weight=5,line_opacity=0.3).add_to(m)

In [74]:
display(m)

***

# 4) Abbiegeverbote Stadt Zürich
<br>Die Abbiegeverbote geben an, wo man als Velofahrerin nicht abbiegen darf.<br/>
<br>Zugriff: 01:11:2021/20:05 https://data.stadt-zuerich.ch/dataset/geo_fuss__und_velowegnetz<br/>

In [75]:
gdf_abbiegeverbot = gpd.read_file('./Daten/Fuss_Velo_Wegnetz/data/taz_mm.tbl_routennetz_abbiegeverbote.json')
gdf_abbiegeverbot.head()

,id1,se_anno_cad_data,objectid,geometry
0,108,NaN,1,"LINESTRING (8.54526 47.36752, 8.54522 47.36750..."
1,109,NaN,2,"LINESTRING (8.54259 47.37874, 8.54255 47.37867)"
2,110,NaN,3,"LINESTRING (8.55336 47.40247, 8.55345 47.40251..."
3,111,NaN,4,"LINESTRING (8.50459 47.40565, 8.50460 47.40563..."
4,112,NaN,5,"LINESTRING (8.53681 47.38516, 8.53679 47.38521..."


In [76]:
gdf_abbiegeverbot.drop(['se_anno_cad_data','id1','objectid'], axis=1, inplace=True)

In [77]:
gdf_abbiegeverbot.head()

,geometry
0,"LINESTRING (8.54526 47.36752, 8.54522 47.36750..."
1,"LINESTRING (8.54259 47.37874, 8.54255 47.37867)"
2,"LINESTRING (8.55336 47.40247, 8.55345 47.40251..."
3,"LINESTRING (8.50459 47.40565, 8.50460 47.40563..."
4,"LINESTRING (8.53681 47.38516, 8.53679 47.38521..."


In [78]:
gdf_abbiegeverbot.to_csv('./Daten/abbiegeverbote_ZH.csv', index=False)